In [ ]:
## imports
import glob, os

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import requests
from io import BytesIO
from PIL import Image
import numpy as np
pylab.rcParams['figure.figsize'] = 20, 12
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.engine.predictor_glip import GLIPDemo

In [ ]:
def load(file):
    pil_image = Image.open(file).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def load_url(url):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img, caption):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.figtext(0.5, 0.09, caption, wrap=True, horizontalalignment='center', fontsize=20)

def imsave(img, path):
    plt.imsave(path, img[:, :, [2, 1, 0]])

In [ ]:
config_file = "configs/pretrain/glip_Swin_L.yaml"
weight_file = "MODEL/glip_large_model.pth"

# update the config options with the config file
# manual override some options
cfg.local_rank = 0
cfg.num_gpus = 1
cfg.merge_from_file(config_file)
cfg.merge_from_list(["MODEL.WEIGHT", weight_file])
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])

glip_demo = GLIPDemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.7,
    show_mask_heatmaps=False
)

In [ ]:
#caption = 'person . bicycle . car . motorcycle . airplane . bus . train . truck . boat . traffic light . fire hydrant . stop sign . parking meter . bench . bird . cat . dog . horse . sheep . cow . elephant . bear . zebra . giraffe . backpack . umbrella . handbag . tie . suitcase . frisbee . skis . snowboard . sports ball . kite . baseball bat . baseball glove . skateboard . surfboard . tennis racket . bottle . wine glass . cup . fork . knife . spoon . bowl . banana . apple . sandwich . orange . broccoli . carrot . hot dog . pizza . donut . cake . chair . couch . potted plant . bed . dining table . toilet . tv . laptop . mouse . remote . keyboard . cell phone . microwave . oven . toaster . sink . refrigerator . book . clock . vase . scissors . teddy bear . hair drier . toothbrush .'
#caption = ['man in back wearing grey shirt . women walking in background . ', 'man in yellow shirt . blue shirt . ']
caption = ['brown tray on the table . ', 'men in far right in blue shirt . men in far right in black shirt . ', 'player in the far left back in red shirt . ']

In [ ]:
#for files in [glob.glob(e) for e in ['INPUT/*.jpg', 'INPUT/*.png', 'INPUT/*.JPG', 'INPUT/*.PNG']]:
# for files in [glob.glob(e) for e in ['INPUT/*.png']]:
files = ['INPUT/couch.png', 'INPUT/gym.png', 'INPUT/players.png']
for i, file in enumerate(files):
    image = load(file)
    result, _ = glip_demo.run_on_web_image(image, caption[i], 0.5)
    #imshow(result, caption)
    imsave(result, "RESULTS/"+file.split("/")[-1])